In [1]:
import requests
import json
import pandas as pd
import numpy as np
import re

## Получаем темы обсуждений

Параметры для формирования запросов

In [2]:
domain = 'podslstankin' # сокращенный адрес группы вк (подслушано СТАНКИН)
count = 100 # количество записей для запроса (максимум 100)
access_token = '' # токен авторизованного приложения
v = '5.124' # версия api вк

Формируем ссылку для запроса и получаем ответ

In [3]:
link_topics = 'https://api.vk.com/method/wall.get?domain={0}&count={1}&access_token={2}&v={3}'.format(domain, count, access_token, v) # ссылка для запроса
request_topics = requests.get(link_topics) # ответ на запрос в формате json
print(link_topics) # сформированная ссылка
#print(request_topics.text) # результат выполнения запроса

https://api.vk.com/method/wall.get?domain=podslstankin&count=100&access_token=52b9157b52b9157b52b9157b2752cd6b0b552b952b9157b0d34dbcc279b15430fcd3ab5&v=5.124


Записываем в переменную количество тем в группе вк

Переменная заводится для того, чтобы при новом запуске парсера вытащить все строки, а не ограничиться конкретным числом (если есть новые сообщения в группе)

In [4]:
pattern_size = r'\{\"response\"\:\{\"count\"\:(.*?)\,'
size = int(re.findall(pattern_size, request_topics.text)[0]) # вытаскиваем из результата запроса количество тем
size

14151

Обрабатываем полученный ответ и получаем списки с ID постов, ID владельца стены, даты и тексты постов. Формируем первые записи для набора данных в датафрейме

In [5]:
def formation_df(request_topics):
    pattern = r'\{\"id(.*?)\}\}'
    items = re.findall(pattern, request_topics.text) # вытаскиваем все данные, относящиеся к теме 
    pattern = r'\"\:(.*?)\,.*?\"owner_id\"\:(.*?)\,\"date\"\:(.*?)\,.*?\"text\"\:\"(.*?)\"\,'
    list_id = [] # id темы
    list_owner_id = [] # id владельца стены
    list_date = [] # дату, когда была опубликована тема
    list_text = [] # текст темы
    for i in items:
        k = re.findall(pattern, i) # вытаскиваем необходимые данные (которые выше) из результата запроса по каждой теме
        for j in k:
            list_id.append(j[0])
            list_owner_id.append(j[1])
            list_date.append(j[2])
            list_text.append(j[3])
    d = {'ID':np.array(list_id), 'OwnerID':np.array(list_owner_id),
                 'Date':np.array(list_date), 'Text':np.array(list_text)} # формируем словарь для записи в виде датафрейма
    df = pd.DataFrame(d) # формируем датафрейм
    return df

Смотрим результат

In [6]:
df = formation_df(request_topics)
df

,ID,OwnerID,Date,Text
0,115862,-58804998,1614801537,Ах вот что это. Анон
1,115861,-58804998,1614801526,Есть фото общежития внутри из Люблино?
2,115860,-58804998,1614801505,У 1 или 2 курса будет обязательная производств...
3,115850,-58804998,1614755304,Потеряла пропуск на имя Марии Бурлаковой.\nНап...
4,115840,-58804998,1614603396,"Анон\nИщу старост групп ИДБ-20-01,02,03,04"
...,...,...,...,...
95,115094,-58804998,1608914149,Преподы отправляют модули после срока\/Деканат...
96,115093,-58804998,1608914138,Найдутся ли у кого билеты за 3 семестр по мате...
97,115077,-58804998,1608816111,"3-е общежитие, умеет ли кто делать капельницы?..."
98,115067,-58804998,1608392190,У кого-нибудь остались лекции по ИЭС Быстрико...


Функция для дозаполнения созданного датафрейма df_old новым df_new, содержащий количество записей offset

In [7]:
def new_dataframe(offset, df_old, domain, count, access_token, v):
    link_topics = 'https://api.vk.com/method/wall.get?domain={0}&offset={1}&count={2}&access_token={3}&v={4}'.format(domain, offset, count, access_token, v)
    request_topics = requests.get(link_topics)
    df_new = formation_df(request_topics)
    df = df_old.append(df_new)
    return df

Запускаем функцию для сбора всех записей в группе

In [8]:
df_old = formation_df(request_topics)
offset = 100
while offset < size:
    df = new_dataframe(offset, df_old, domain, count, access_token, v)
    df_old = df
    offset += 100
    
df

,ID,OwnerID,Date,Text
0,115862,-58804998,1614801537,Ах вот что это. Анон
1,115861,-58804998,1614801526,Есть фото общежития внутри из Люблино?
2,115860,-58804998,1614801505,У 1 или 2 курса будет обязательная производств...
3,115850,-58804998,1614755304,Потеряла пропуск на имя Марии Бурлаковой.\nНап...
4,115840,-58804998,1614603396,"Анон\nИщу старост групп ИДБ-20-01,02,03,04"
...,...,...,...,...
46,17,-58804998,1380041274,Это особенность такая у студентов технических ...
47,15,-58804998,1380041060,"Другу понравилась наша староста, но она помойм..."
48,10,-58804998,1380037340,Позитивно:)
49,7,-58804998,1380036426,"Меня очень удивило, как перваки сегодня стоят ..."


Записываем датафрейм в файл themes_raw.csv

In [9]:
df.to_csv('themes_raw.csv')

## Обработка данных с темами обсуждений

Создаем dataframe из созданного файла, чтобы в случае ошибок не было необходимо заново собирать данные через api.vk

In [10]:
df = pd.read_csv('themes_raw.csv')
df

,Unnamed: 0,ID,OwnerID,Date,Text
0,0,115862,-58804998,1614801537,Ах вот что это. Анон
1,1,115861,-58804998,1614801526,Есть фото общежития внутри из Люблино?
2,2,115860,-58804998,1614801505,У 1 или 2 курса будет обязательная производств...
3,3,115850,-58804998,1614755304,Потеряла пропуск на имя Марии Бурлаковой.\nНап...
4,4,115840,-58804998,1614603396,"Анон\nИщу старост групп ИДБ-20-01,02,03,04"
...,...,...,...,...,...
14151,46,17,-58804998,1380041274,Это особенность такая у студентов технических ...
14152,47,15,-58804998,1380041060,"Другу понравилась наша староста, но она помойм..."
14153,48,10,-58804998,1380037340,Позитивно:)
14154,49,7,-58804998,1380036426,"Меня очень удивило, как перваки сегодня стоят ..."


In [11]:
df = df.drop('Unnamed: 0', axis=1)
df

,ID,OwnerID,Date,Text
0,115862,-58804998,1614801537,Ах вот что это. Анон
1,115861,-58804998,1614801526,Есть фото общежития внутри из Люблино?
2,115860,-58804998,1614801505,У 1 или 2 курса будет обязательная производств...
3,115850,-58804998,1614755304,Потеряла пропуск на имя Марии Бурлаковой.\nНап...
4,115840,-58804998,1614603396,"Анон\nИщу старост групп ИДБ-20-01,02,03,04"
...,...,...,...,...
14151,17,-58804998,1380041274,Это особенность такая у студентов технических ...
14152,15,-58804998,1380041060,"Другу понравилась наша староста, но она помойм..."
14153,10,-58804998,1380037340,Позитивно:)
14154,7,-58804998,1380036426,"Меня очень удивило, как перваки сегодня стоят ..."


### Анализ датафрейма по колонке ID

In [12]:
df['ID'].describe()

count              14156
unique             14113
top       {"type":"mvk"}
freq                  28
Name: ID, dtype: object

Каждый ID должен быть уникальным, однако некоторые являются строкой "{"type":"mvk"}", что не соответствует формату

In [13]:
df[df['ID'] == '{"type":"mvk"}']

,ID,OwnerID,Date,Text
3935,"{""type"":""mvk""}",-58804998,1530047229,"День добрый, дамы и господа. Рассматриваю вари..."
5698,"{""type"":""mvk""}",-58804998,1503575766,Для заселения в общежитие нужно ли сниматься с...
8881,"{""type"":""mvk""}",-58804998,1449581131,"Оставил очки где-то. \nЧерная оправа, с красны..."
8903,"{""type"":""mvk""}",-58804998,1449234292,Примет ли веселко коньяк за защиту лабораторно...
9992,"{""type"":""mvk""}",-58804998,1439489622,Что лучше использовать тетрадки или блоки?(ФЭМ...
10024,"{""type"":""mvk""}",-58804998,1439116385,"Группа ИДБ-12-13, откликнитесь пожалуйста комм..."
10933,"{""type"":""mvk""}",-58804998,1420546052,"ребят, январская стипуха уже от рейтинга завис..."
11001,"{""type"":""mvk""}",-58804998,1419407590,"Господа, что делать, если не можешь сдать алге..."
11092,"{""type"":""mvk""}",-58804998,1418424725,Давайте скинемся на пулю для Лакуниной!\n_____...
11130,"{""type"":""mvk""}",-58804998,1417973509,Есть у кого-нибудь клетка или аквариум для чер...


Удаляем строки с найденными индексами

In [14]:
index_list_to_del = df.index[df['ID'] == '{"type":"mvk"}'].tolist() # строки для удаления с неправильным ID темы
df = df.drop(index_list_to_del)
df['ID'].describe()

count             14128
unique            14112
top       {"type":"api"
freq                  9
Name: ID, dtype: object

Удаляем следующие строки с неправильным форматом ID

In [15]:
index_list_to_del = df.index[df['ID'] == '{"type":"api"'].tolist() # строки для удаления с неправильным ID темы
df = df.drop(index_list_to_del)
df['ID'].describe()

count             14119
unique            14111
top       {"type":"vk"}
freq                  9
Name: ID, dtype: object

In [16]:
index_list_to_del = df.index[df['ID'] == '{"type":"vk"}'].tolist() # строки для удаления с неправильным ID темы
df = df.drop(index_list_to_del)
df['ID'].describe()

count     14110
unique    14110
top       21344
freq          1
Name: ID, dtype: object

Теперь все ID являются уникальными, значит удаление лишних строк по колонке ID завершена

### Анализ датафрейма по колонке OwnerID

In [17]:
df['OwnerID'].describe()

count         14110
unique            4
top       -58804998
freq          14105
Name: OwnerID, dtype: object

In [18]:
df[df['OwnerID'] != '-58804998']

,ID,OwnerID,Date,Text
170,"[{""id"":166","-143144642,""from_id"":-143144642",1603735207,"29 октября в МГТУ \""СТАНКИН\"" состоятся открыт..."
2674,"[{""id"":944","-88941246,""from_id"":-88941246",1547834958,❗️❕Внимание❕❗️\n🚗Набор в автошколу! 🚗\n🚗🏍Катег...
3682,"[{""id"":4868","-122997027,""from_id"":-122997027",1533744999,NaN
4257,"[{""id"":3388","-122997027,""from_id"":-122997027",1522010001,анон
5363,"[{""id"":892","-122997027,""from_id"":-122997027",1506156720,Отряд самоубийц


Темы, которые публиковались другими пользователями также имеют немного неправильный ID (которые можно поправить), но для простоты дальнейшей работы эти записи стоит удалить

In [19]:
index_list_to_del = df.index[df['OwnerID'] != '-58804998'].tolist() # строки для удаления с неправильным ID темы
df = df.drop(index_list_to_del)
df['OwnerID'].describe()

count         14105
unique            1
top       -58804998
freq          14105
Name: OwnerID, dtype: object

Существует всего одно уникальное значение OwnerID, значит удаление лишних строк по колонке OwnerID завершена

### Проверка на оставшиеся строчки, содержащие неверные форматы в колонках

Проверим типы данных колонок в датафрейме

In [20]:
df.dtypes

ID         object
OwnerID    object
Date        int64
Text       object
dtype: object

Колонки ID и OwnerID не являются числовыми данными, исправим это

In [21]:
df['ID'] = pd.to_numeric(df['ID'])
df['OwnerID'] = pd.to_numeric(df['OwnerID'])
df.dtypes

ID          int64
OwnerID     int64
Date        int64
Text       object
dtype: object

Все необходимые колоники имеют верный формат, а значит правильное содержимое (согласно анализу и приведение к формату int без ошибок)

### Анализ текстов тем обсуждений

In [22]:
df.dropna(inplace=True) # удаляем все строки, в которых пропущены значения
df = df.reset_index(drop=True) # делаем переиндексацию после удаленных строк

Функция для поиска индексов слов с количеством пробелов j в датафрейме df

In [23]:
def space_count(j, df):
    i = 0
    l = []
    while i < len(df):
        if df.loc[i, 'Text'].count(' ') <= j:
            l.append(i)
        i += 1
    return l

In [24]:
l = space_count(0, df) # без пробелов
df.loc[l]

,ID,OwnerID,Date,Text
7,115812,-58804998,1614372815,анон
506,111820,-58804998,1596287463,Разделение
682,110469,-58804998,1584470227,гвоздкова
768,109925,-58804998,1581944412,Зачем?
1053,108298,-58804998,1575033095,Wtf?!
...,...,...,...,...
12516,12595,-58804998,1397850776,доставляет)
12610,11738,-58804998,1396866886,Ждём?
12857,9738,-58804998,1394427937,[id163198870|МАААААААААААКС!]
12977,8568,-58804998,1393149550,Норм👍


In [25]:
l = space_count(1, df) # один пробел
df.loc[l]

,ID,OwnerID,Date,Text
7,115812,-58804998,1614372815,анон
13,115756,-58804998,1613842126,Долой интернетушку?
47,115512,-58804998,1612511163,"Тамази, F"
228,114082,-58804998,1601726304,Потеряны наушники
265,113822,-58804998,1601055429,Чьё добро?
...,...,...,...,...
13094,7462,-58804998,1391795251,Увидел расписание.
13099,7436,-58804998,1391789284,О наболевшем
13545,3805,-58804998,1385934032,http:\/\/vk.com\/fuck_humor?z=photo-12382740_3...
13785,1365,-58804998,1383603902,Заебали бухать.


In [26]:
l = space_count(2, df) # два пробела
df.loc[l]

,ID,OwnerID,Date,Text
7,115812,-58804998,1614372815,анон
13,115756,-58804998,1613842126,Долой интернетушку?
30,115611,-58804998,1612862998,"Ищу старост ИДБ-19-9,10,11"
47,115512,-58804998,1612511163,"Тамази, F"
49,115501,-58804998,1612454897,Завтра работает библиотека?\nАнон
...,...,...,...,...
13661,2735,-58804998,1384966045,почему Феофан бухает?
13768,1463,-58804998,1383738367,"Елькин круче В.Б.,извините)"
13785,1365,-58804998,1383603902,Заебали бухать.
13837,797,-58804998,1382567401,кто такой Носовицкий?


Удаляем все строки, в которых находятся меньше одного пробела (меньше трех слов), так как они скорее всего содержат медиафайлы или ссылки, по которым не получится оценивать деструктивность темы

In [27]:
l1 = space_count(0, df)
l2 = space_count(1, df)
l = l1 + l2
df = df.drop(l)
print('Удалено', len(l), 'элементов')

Удалено 220 элементов


In [28]:
print('В результате предварительной чистки данных удалено ', size - len(df), ' тем ', '(', round((size - len(df)) / size * 100, 2), '% от всех данных)', sep='')

В результате предварительной чистки данных удалено 347 тем (2.45% от всех данных)


Записываем датафрейм в файл themes_processed.csv

In [29]:
df = df.reset_index(drop=True) # делаем переиндексацию после удаленных строк
df.to_csv('themes_processed.csv')

## Получаем комментарии для темы обсуждения 

In [30]:
owner_id = -58804998 # id владельца сообщества
thread_items_count = 10 # количество ответов к комментарию пользователя (10 максимум)

Функция для получения комментариев к посту с id "post_id"

In [31]:
link_comments = 'https://api.vk.com/method/wall.getComments?owner_id={0}&post_id={1}&count={2}&thread_items_count={3}&access_token={4}&v={5}'.format(owner_id, 115136, count, thread_items_count, access_token, v)
request_comments = requests.get(link_comments)

In [32]:
def collecting_comments(post_id, owner_id, count, thread_items_count, access_token, v):
    link_comments = 'https://api.vk.com/method/wall.getComments?owner_id={0}&post_id={1}&count={2}&thread_items_count={3}&access_token={4}&v={5}'.format(owner_id, post_id, count, thread_items_count, access_token, v)
    request_comments = requests.get(link_comments)
    #pattern_number_of_comments = r'\{\"response\"\:\{\"count\"\:(.*?)\,'
    #number_of_comments = int(re.findall(pattern_size, request_comments.text)[0]) # вытаскиваем из результата запроса количество тем
    pattern_texts = r'\"text\"\:\"(.*?)\"'
    texts = re.findall(pattern_texts, request_comments.text)
    l = []
    for i in texts:
        if i != '':
            a = re.sub(r'\[.*\|.*\]\, ', '', i)
            if a.count(' ') >= 1: # добавляем комментарий, если в нем содержится 2 и более слов
                l.append(a)
    return l

In [33]:
collecting_comments(115136, owner_id, count, thread_items_count, access_token, v)

['Следующую сессию вовремя закрываешь с общим рейтингом больше 35 и на здоровье.',
 'а что делать если последний семестр?',
 'если последний, то ещё в июле и августе можно получить стипендию (если остаёшься в нашу магистратуру или если никуда больше поступать не планируешь)',
 'понимаю твою боль']

Открываем обработанный файл с темами и добавляем колонку Count_comments, содержащую количество комментариев в теме

In [34]:
df = pd.read_csv('themes_processed.csv')
df = df.drop('Unnamed: 0', axis=1)
df['Count_comments'] = 0

Удалим значения ID, возвращающие ошибку при получении запроса

In [35]:
index_list_to_del = df.index[df['ID'] > 1000000].tolist() # строки для удаления с неправильным ID темы
df = df.drop(index_list_to_del)
df = df.reset_index(drop=True) # делаем переиндексацию после удаленных строк

Считаем количество комментариев для каждой темы и создаём датафрейм df_comments, содержащий тексты комментариев и id темы, к которым они относятся

In [36]:
i = 0
list_id = []
list_text = []
while i < len(df):
    post_id = df.loc[i, 'ID']
    #print(post_id, round(i / len(df) * 100, 2))
    texts = collecting_comments(post_id, owner_id, count, thread_items_count, access_token, v)
    df.loc[i, 'Count_comments'] = len(texts)
    for j in texts:
        list_id.append(post_id)
        list_text.append(j)
    i += 1
d = {'Post_ID':np.array(list_id), 'Text':np.array(list_text)}
df_comments = pd.DataFrame(d)
df_comments.to_csv('comments_raw.csv')

In [37]:
df_comments

,Post_ID,Text
0,115862,"Да, я тоже сначала не понял"
1,115862,Эталон гармонии!
2,115862,звучит хайпово
3,115861,В гугле
4,115861,Ну вообще тут нормально
...,...,...
61743,4,"А начать с ней общаться, не? (="
61744,4,Вадим Борисович. Вы все комментарии читаете ил...
61745,4,привет из 2016
61746,4,привет из 2019


In [38]:
df

,ID,OwnerID,Date,Text,Count_comments
0,115862,-58804998,1614801537,Ах вот что это. Анон,3
1,115861,-58804998,1614801526,Есть фото общежития внутри из Люблино?,14
2,115860,-58804998,1614801505,У 1 или 2 курса будет обязательная производств...,1
3,115850,-58804998,1614755304,Потеряла пропуск на имя Марии Бурлаковой.\nНап...,0
4,115840,-58804998,1614603396,"Анон\nИщу старост групп ИДБ-20-01,02,03,04",1
...,...,...,...,...,...
13759,20,-58804998,1380042205,а мы счастливы вместе :) кто бы мог подумать :...,3
13760,17,-58804998,1380041274,Это особенность такая у студентов технических ...,3
13761,15,-58804998,1380041060,"Другу понравилась наша староста, но она помойм...",4
13762,7,-58804998,1380036426,"Меня очень удивило, как перваки сегодня стоят ...",2


In [39]:
df.to_csv('themes_processed.csv')